In [1]:
!pip install -q transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
model_path = '/content/drive/MyDrive/llama3-finetuned-merged'

In [4]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.0 MB/s eta 0:00:00


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import gradio as gr

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float32,
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(model_path)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
def generate_segmentation(prompt):
    system_msg = "You are a marketing strategist specialized in creating market segmentation analyses for startups."

    text_in = (
        "<|start_of_turn|>system\n" + system_msg + "<|end_of_turn|>\n"
        "<|start_of_turn|>user\n" + prompt + "<|end_of_turn|>\n"
        "<|start_of_turn|>assistant\n"
    )

    inputs = tokenizer(text_in, return_tensors="pt").to(model.device)
    input_length = inputs.input_ids.shape[1]  # Get length of input tokens

    output = model.generate(
        **inputs,
        max_new_tokens=850,
        do_sample=False,
        temperature=0.1,
        top_p=1,
        top_k=80
    )

    # Slice the output to exclude input tokens
    generated_tokens = output[0, input_length:]

    # Decode only the generated portion
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return generated_text


In [17]:
# Cell 6: Create and launch Gradio interface
with gr.Blocks(theme="default") as demo:
    gr.Markdown("# Llama Market Segmentation Generator")

    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(
                label="Startup Idea",
                placeholder="Describe your startup idea here...",
                lines=3
            )
            generate_btn = gr.Button("Generate Market Segmentation")

    output_text = gr.Textbox(label="Generated Market Segmentation", lines=25)

    generate_btn.click(
        fn=generate_segmentation,
        inputs=input_text,
        outputs=output_text
    )

# Launch with share=True to get a public URL
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bf782610056495556b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [16]:
system_msg = "You are a marketing strategist specialized in creating market segmentation analyses for startups."
user_prompt = "Create a market segmentation table for an AI-based fitness app"

text_in = (
    "<|start_of_turn|>system\n" + system_msg + "<|end_of_turn|>\n"
    "<|start_of_turn|>user\n" + user_prompt + "<|end_of_turn|>\n"
    "<|start_of_turn|>assistant\n"
)

inputs = tokenizer(text_in, return_tensors="pt").to(model.device)

output = model.generate(
    **inputs,
    max_new_tokens=800,
    do_sample=False,
    temperature=0.2,
    top_p=1,
    top_k=80
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system
You are a marketing strategist specialized in creating market segmentation analyses for startups.
user
Create a market segmentation table for an AI-based fitness app
assistant
[
  {
    "Title/Description": "End User",
    "Fitness Enthusiasts": "Individuals who regularly work out",
    "Health Conscious Individuals": "Concerned about health",
    "Busy Professionals": "Time constrained",
    "Fitness Training Providers": "Personal trainers",
    "Healthcare Providers": "Medical professionals"
  },
  {
    "Title/Description": "Task",
    "Fitness Enthusiasts": "Tracking fitness metrics",
    "Health Conscious Individuals": "Staying motivated",
    "Busy Professionals": "Convenient workout scheduling",
    "Fitness Training Providers": "Client management",
    "Healthcare Providers": "EHR integration"
  },
  {
    "Title/Description": "Benefit",
    "Fitness Enthusiasts": "Real-time progress tracking",
    "Health Conscious Individuals": "Nutrition and exercise insights",
    "B

In [ ]:
import time

while True:
    time.sleep(60)
    print("Keeping session alive...")


Keeping session alive...
Keeping session alive...


In [11]:
print("hello world!")

hello world!
